In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers

import matplotlib.pyplot as plt

MODELS= ['AT_1', 'AT_2', 'AT_3']
#Shift = 22 means 30 seconds after, -22 is 30 seconds before (aprox)
SHIFT= 22
RATIO= 80.0/65.0
MEAN_TEMP= 67.872055
MEAN_TIME= 0.536641
STD_TEMP= 14.012810
STD_TIME= 0.069499
ds= {}

#Load data
raw_dataset= pd.read_csv('data.csv', na_values='?')
raw_dataset.pop('A')

#Preprocess data
for m in MODELS: 
    ds[m]= raw_dataset.copy()
    ds[m]= ds[m].drop(ds[m][ds[m]['model'] != m].index)
    ds[m].pop('model')
    ds[m]['Ti']= ds[m]['T'].shift(SHIFT)
    ds[m]['min']= ds[m]["min"].rolling(window=SHIFT, min_periods=1).sum()
    ds[m]= ds[m].dropna() 
ds_t= pd.concat(ds)
ds_t['Ti']= ds_t['Ti'] * RATIO
ds_t['T']= ds_t['T'] * RATIO
print(ds_t.head())

            min          T         Ti
AT_3 4656  0.52  33.550769  31.347692
     4657  0.52  33.624615  31.347692
     4658  0.52  33.710769  31.335385
     4659  0.50  33.796923  31.384615
     4660  0.52  33.883077  31.532308


In [3]:

#Split tran and test sets
train= ds_t.sample(frac=0.8, random_state=0)
test= ds_t.drop(train.index)

#Get data stats
train_stats = train.describe()
train_stats.pop('T')
train_stats = train_stats.transpose()

#Pop the output from the dataset
train_labels = train.pop('T')
test_labels = test.pop('T')

#Normalize data
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train)
normed_test_data = norm(test)

#Build model
def build_model():
  model = keras.Sequential([
    layers.Dense(250, activation='relu', input_shape=[len(train.keys())]),
    layers.Dense(250, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)
  model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
  
  return model

model = build_model()
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

#Run the model
EPOCHS = 1000
history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[early_stop])

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

c:\Users\Josè Castillo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


,loss,mae,mse,val_loss,val_mae,val_mse,epoch
40,0.918749,0.775378,0.918749,0.305087,0.522991,0.305087,40
41,0.660789,0.689697,0.660789,0.697892,0.651103,0.697892,41
42,0.857136,0.811509,0.857136,1.134641,1.028511,1.134641,42
43,0.748190,0.617129,0.748190,0.103083,0.296344,0.103083,43
44,0.941755,0.825260,0.941755,0.365323,0.563294,0.365323,44


In [4]:
def cold(t: float):
    t= (33.009231 - MEAN_TEMP) / STD_TEMP
    m= (0.51 - MEAN_TIME)/ STD_TIME
    return model.predict(pd.DataFrame({'min': [m], 'Ti': [t]})).flatten()

print(cold(33.009231))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
[33.74002]


In [ ]:
print((34.412308 - MEAN_TEMP) / STD_TEMP, (0.51 - MEAN_TIME)/ STD_TIME)

In [ ]:
print(normed_test_data.head())
print(test.head())
print(model.predict(normed_test_data).flatten())


In [ ]:
#Test the model
test_predictions = model.predict(normed_test_data).flatten()

print(test_predictions)

In [ ]:
print(test_labels)

In [5]:
model.save('models/warmAto.keras')